In [1]:
#import the relevant modules to read from .mat file and to manipulate NWB file groups and datasets

import numpy as np
import datetime
from dateutil.tz import tzlocal
import pytz
import h5py
from hdmf.backends.hdf5 import H5DataIO
import hdf5storage
import os
from pynwb import NWBFile, NWBHDF5IO, ProcessingModule
from pynwb.device import Device
from pynwb.base import TimeSeries
from pynwb.ecephys import ElectrodeGroup, SpikeEventSeries

In [2]:
#Load the .mat files containing sorted spikes

path_to_files = 'C:/Users/Admin/Desktop/Ben Dichter/nwbn-conversion-tools/datafiles'

# Open info file
fname0 = 'indy_20160407_02.mat'
fpath0 = os.path.join(path_to_files, fname0)
f_info = hdf5storage.loadmat(fpath0)
#f_info = h5py.File(fpath0)
info = f_info.keys()
#f_info['spikes']

In [3]:
# Create a new NWB file instance

session_start_time = datetime.datetime(2016,4,7,tzinfo=pytz.timezone("America/Los_Angeles"))
experiment_description = 'The behavioral task was to make self-paced reaches to targets arranged in a grid (e.g. 8x8) without gaps or pre-movement delay intervals.'

nwb = NWBFile(session_description='Multichannel Sensorimotor Cortex Electrophysiology', 
              identifier='indy_20160407_02', 
              session_start_time=session_start_time,
              experimenter='Joseph E. ODoherty',
              lab='Sabes lab',
              institution='University of California, San Francisco',
              experiment_description='experiment_description',
              session_id='indy_20160407_02')
print(nwb)

root pynwb.file.NWBFile at 0x2309406322248
Fields:
  experiment_description: experiment_description
  experimenter: ['Joseph E. ODoherty']
  file_create_date: [datetime.datetime(2020, 4, 1, 13, 2, 51, 684318, tzinfo=tzlocal())]
  identifier: indy_20160407_02
  institution: University of California, San Francisco
  lab: Sabes lab
  session_description: Multichannel Sensorimotor Cortex Electrophysiology
  session_id: indy_20160407_02
  session_start_time: 2016-04-07 00:00:00-07:53
  timestamps_reference_time: 2016-04-07 00:00:00-07:53



In [4]:
# Create Device and ElectrodeGroup and adding electrode information to nwb.

#Create and add devices
device_M1 = Device('96_Channels_Electrode_Array_Recording_Device_M1')
nwb.add_device(device_M1)

device_S1 = Device('96_Channels_Electrode_Array_Recording_Device_S1')
nwb.add_device(device_S1)

# Create Electrode group M1
electrode_group_M1 = ElectrodeGroup(name='ElectrodeArray_M1', description='96 Channels Array', 
                                    location='Motor Cortex', 
                                    device=device_M1)

for idx in np.arange(96):
    nwb.add_electrode(id=idx,
                          x=np.nan, y=np.nan, z=np.nan,
                          imp=float(-idx),
                          location='M1', filtering='none',
                          group=electrode_group_M1)


# Create Electrode group S1
electrode_group_S1 = ElectrodeGroup(name='ElectrodeArray_S1', description='96 Channels Array', 
                                    location='Somatosensory Cortex', 
                                    device=device_S1)

for idx in np.arange(96):
    nwb.add_electrode(id=idx,
                          x=np.nan, y=np.nan, z=np.nan,
                          imp=float(-idx),
                          location='S1', filtering='none',
                          group=electrode_group_S1)


print(nwb)

root pynwb.file.NWBFile at 0x2309406322248
Fields:
  devices: {
    96_Channels_Electrode_Array_Recording_Device_M1 <class 'pynwb.device.Device'>,
    96_Channels_Electrode_Array_Recording_Device_S1 <class 'pynwb.device.Device'>
  }
  electrodes: electrodes <class 'hdmf.common.table.DynamicTable'>
  experiment_description: experiment_description
  experimenter: ['Joseph E. ODoherty']
  file_create_date: [datetime.datetime(2020, 4, 1, 13, 2, 51, 684318, tzinfo=tzlocal())]
  identifier: indy_20160407_02
  institution: University of California, San Francisco
  lab: Sabes lab
  session_description: Multichannel Sensorimotor Cortex Electrophysiology
  session_id: indy_20160407_02
  session_start_time: 2016-04-07 00:00:00-07:53
  timestamps_reference_time: 2016-04-07 00:00:00-07:53



In [5]:
# Create SpikeEventSeries container to store the spike waveform data. Spike waveform data usually goes on the acquisition group of NWB files.

#Stores raw data in acquisition group

electrode_table_region_M1 = nwb.create_electrode_table_region(list(range(96)), 'the 96 electrodes in the array M1')
description = 'A cell array of spike event waveform "snippets". Each element in the cell array is a matrix of spike event waveforms. Each waveform corresponds to a timestamp in "spikes". Waveform samples are in microvolts.'
ephys_ts_M1 = SpikeEventSeries(name='M1 Spike Events',
                            data=f_info['wf'][0:96,:],
                            timestamps=f_info['spikes'][0:96,:],
                            electrodes=electrode_table_region_M1,
                            resolution=4.096e-05,
                            conversion=np.nan,
                            description=description)

nwb.add_acquisition(ephys_ts_M1)


electrode_table_region_S1 = nwb.create_electrode_table_region(list(range(96)), 'the 96 electrodes in the array S1')
description = 'A cell array of spike event waveform "snippets". Each element in the cell array is a matrix of spike event waveforms. Each waveform corresponds to a timestamp in "spikes". Waveform samples are in microvolts.'
ephys_ts_S1 = SpikeEventSeries(name='S1 Spike Events',
                            data=f_info['wf'][96:192,:],
                            timestamps=f_info['spikes'][96:192,:],
                            electrodes=electrode_table_region_S1,
                            resolution=4.096e-05,
                            conversion=np.nan,
                            description=description)

nwb.add_acquisition(ephys_ts_S1)

print(nwb.acquisition)

ValueError: ElectricalSeries.__init__: incorrect shape for 'timestamps' (got '(96, 3)', expected '(None,)')

In [ ]:
# Associate electrodes with units


In [ ]:
# Save NWB to file:

fname_nwb = 'indy_20160407_02.nwb'
fpath_nwb = os.path.join(path_to_files, fname_nwb)
with NWBHDF5IO(fpath_nwb, mode='w') as io:
    io.write(nwb)
print('File saved with size: ', os.stat(fpath_nwb).st_size/1e6, ' mb')

In [ ]:
# Load NWB:

io = NWBHDF5IO(fpath_nwb, mode='r')
nwbfile = io.read()